</div>
<div align="center">
  <img src="img/evolnode.png" width="800" alt="Fourier reconstruction convergence">
  <p><em> Evolution Node free you from coding and debugging, let LLM evolve your code for you.</em></p>
</div>
<div align="left">
<p><em>Bored of manual coding, a function? EvolNode let LLM automate function design and guide the evolution of it with genetic algorithm. A node here takes a task, input, and output, it uses either code or another LLM to complete the task, ensuring aligned input and output value types and ,.names.Fitness evaluation is done by running the function with a few specified test cases, the more diverse the test case, the better the evolution.
</em></p>
</div>

In [3]:
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.eoh_evolution import EvolNode


# Code + Compilor Task
# mp = MetaPrompt("Generate Fibonacci sequence.", "fibonacci", ["n"], ["sequence"], ["int"], ["list"], PromptMode.CODE) # 
# node = EvolNode(mp, None, None)
# input_dict = {"n": 10}
# reasoning, code = node.evolve([input_dict], "i1", replace=True) # Evolution with guaranteed structural fitness
# node(input_dict) # Ok we need a output dictionary here as well ...


# Prompt + LLM Task
mp = MetaPrompt("Get the age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.PROMPT) # 
node = EvolNode(mp, None, None)
input_dict = {"name": "Dilireba"}
reasoning, code = node.evolve([input_dict], "i1", replace=True)
node(input_dict)

KeyboardInterrupt: 


</div>
<div align="center">
  <img src="https://github.com/user-attachments/assets/af98faeb-66d6-4278-af86-67d668d1954e" width="1000" alt="Fourier reconstruction convergence">
  <p><em> But how do we know what are the tasks suitable for our goal? Design of tasks topology is the fundation of planning, let's ask LLM for help on this, too! Evolution Graph autuomate planning by imagning topology of tasks which works best for your goal.</em></p>
</div>


In [ ]:
from methods.meta_prompt import MetaPlan, extract_json_from_text

mp = MetaPlan("Help me date Dilireba, I'm a white colar worker living on salary.")
prompt = mp._get_prompt_i1()
print("MetaPrompt I1: ")
print(prompt)

from methods.llm import get_openai_response
response = get_openai_response(prompt)


MetaPrompt I1: 
Goal: Help me date Dilireba, I'm a white colar worker living on salary.
First, describe the intuition for your tactics and main steps in one sentence. The description must be inside a brace.
Generate a JSON-style plan represented as a Directed Acyclic Graph (DAG) to achieve the goal. Use creative topology in the DAG, include parallel tasks if required.

The plan should include:

- **Nodes**: Each node represents a key action or step and must contain the following attributes:
  - `task`: Description of the task.
  - `name`: Concise name used for the task function.
  - `input`: The resources, information, or prerequisites needed to perform the action.
  - `output`: The immediate result or outcome of the action.
  - `target`: The purpose or goal that the action contributes to.
  - `mode`: The execution mode for this task ("CODE" or "PROMPT").

- **Edges**: Each edge represents a dependency or relationship between nodes, indicating that one step supports or leads to another

In [ ]:
import re 
# from methods.meta_prompt import parse_json_from_response

tactic = re.findall(r"\{(.*)\}", response, re.DOTALL)
plan_dict = extract_json_from_text(response)

In [ ]:
from methods.meta_prompt import MetaPrompt, MetaPlan, extract_json_from_text
from methods.eoh_evolution import EvolNode
from methods.llm import get_openai_response as get_response
from typing import Optional, List, Dict

# Collect MetaPrompt from parsed Plan-Graph Response
meta_node_prompts = []
for node in plan_dict["nodes"]:
    node_prompt = MetaPrompt(task=node.get("task"),  func_name=node.get("name"), input=node.get("input"), output=node.get("output"), mode=node.get("mode").lower())
    meta_node_prompts.append(node_prompt)
    
edges = plan_dict["edges"]

EvolNode(meta_prompt = node_prompt)


#### RepoRAG
* Smooth Dependency Animation

In [1]:
from tools.repo import *

# (I). Pick top growing repo
# fastest_repos = get_fastest_growing_repos(days_ago=14, top_n = 50, print=False)
# repo_url = fastest_repos[0]['html_url']

# (I). Build a GIF of file-dependency of a git repo
repo_url = "https://github.com/xjdr-alt/entropix.git"
temp_repo = "entropix_repo"

file_graph = create_gif_from_repo(repo_url, temp_repo, frame_count=60, fps = 10, output_name="entropix_repo_evolution")

# (II). Build function-level dependency graph of a module within the repo


Repo already cloned, skipping cloning...


success: successfully compiled d2_output/evolve_graph_0.d2 to d2_output/evolve_graph_0.png in 688.717624ms
success: successfully compiled d2_output/evolve_graph_1.d2 to d2_output/evolve_graph_1.png in 551.04575ms
success: successfully compiled d2_output/evolve_graph_2.d2 to d2_output/evolve_graph_2.png in 530.829375ms
success: successfully compiled d2_output/evolve_graph_3.d2 to d2_output/evolve_graph_3.png in 526.930417ms
success: successfully compiled d2_output/evolve_graph_4.d2 to d2_output/evolve_graph_4.png in 528.646584ms
success: successfully compiled d2_output/evolve_graph_5.d2 to d2_output/evolve_graph_5.png in 573.277125ms
success: successfully compiled d2_output/evolve_graph_6.d2 to d2_output/evolve_graph_6.png in 524.445167ms
success: successfully compiled d2_output/evolve_graph_7.d2 to d2_output/evolve_graph_7.png in 505.692208ms
success: successfully compiled d2_output/evolve_graph_8.d2 to d2_output/evolve_graph_8.png in 511.190582ms
success: successfully compiled d2_outp

In [1]:
# (II). Build function-level dependency graph of a module within the repo
from tools.repo import *

temp_repo = "hdcnn" # Contains an interesting new DL model architecture's implementation

python_files = get_python_files(temp_repo) # get all python files 

start_file = python_files[0] # pick the first one for debugging purpose 

dag = build_cross_file_dag(temp_repo, start_file)

name_map = {dag[k]["name"]: k for k in dag} # Map name to node-id for all nodes in the DAG

pick_object = list(name_map.keys())[0] # pick the first object from 'start_file'

sub_dag = extract_subgraph_dag(dag, name_map[pick_object], depth=6) # Extact depedency graph starting from pick_object

sub_dag = decide_opacity_of_dag(sub_dag, progress=1.0, cap_node_number=30) # limit number of nodes for easy understanding

# func_graph = create_evolution_gif(sub_dag, frame_count=60, fps=10) # Sick Animation once again ...
# visualize_dag(sub_dag)

  Number of objects in the subgraph: 15


In [2]:
from methods.llm import get_claude_response # <-- this is the LLM we will be using for summarization
from tools.repo import read_node_content, get_summary_and_code

node_content = read_node_content(sub_dag['node9']) # read code-snippet of a specific node

summary_str, code_str = get_summary_and_code(sub_dag['node9'], get_claude_response) # get summary and code for a specific node

Found class method with name:  HDComputing::superpose
def superpose(self, hvs):
    sum_hv = np.sum(hvs, axis=0)
    return np.sign(sum_hv)


In [11]:
# Bottom-up Summary + Minimal Code ()
from tools.repo import bottom_up_summarization

# Use the pipeline
summarized_dag = bottom_up_summarization(sub_dag, get_claude_response)

# Print summaries and minimal code for each node
for node_id, node in summarized_dag.items():
    print(f"Node: {node['name']} (Level: {node['level']})")
    print(f"Summary: {node['summary']}")
    print(f"Minimal Code:\n{node['minimal_code']}")
    print("-" * 50)

Found function with name:  encode_sequence
Found class method with name:  AGNewsDataset::__getitem__
Found class method with name:  HDComputing::bind
Found function with name:  create_token_hvs
Found class method with name:  AGNewsDataset::__len__
Found class method with name:  HDComputing::random_hv
Found function with name:  build_vocab
Found class method with name:  HDCNNClassifier::forward
Found class method with name:  HDComputing::superpose
Found class method with name:  HDComputing::permute
Found class with name:  AGNewsDataset
Found class with name:  HDComputing
Found function with name:  main
Found class with name:  HDCNNClassifier
Node: AGNewsDataset (Level: 2)
Summary: This code defines a custom PyTorch Dataset class called AGNewsDataset for processing text data. It tokenizes and encodes text into high-dimensional vectors using Hyperdimensional Computing techniques. The class prepares data for machine learning tasks, particularly for text classification, by converting text a

In [12]:
summarized_dag

{'node12': {'name': 'AGNewsDataset',
  'type': 'class',
  'file': 'hdcnn.py',
  'importance': 4,
  'file_path': 'hdcnn/hdcnn.py',
  'edges': ['node3', 'node15', 'node14'],
  'opacity': 0.9333333333333333,
  'level': 2,
  'summary': 'This code defines a custom PyTorch Dataset class called AGNewsDataset for processing text data. It tokenizes and encodes text into high-dimensional vectors using Hyperdimensional Computing techniques. The class prepares data for machine learning tasks, particularly for text classification, by converting text and labels into tensor format.',
  'minimal_code': "from torch.utils.data import Dataset\nfrom nltk.tokenize import word_tokenize\nimport torch\n\nclass AGNewsDataset(Dataset):\n    def __init__(self, data, vocab, token_hvs, hd, max_seq_len, stop_words):\n        self.data = data\n        self.vocab = vocab\n        self.token_hvs = token_hvs\n        self.hd = hd\n        self.max_seq_len = max_seq_len\n        self.stop_words = stop_words\n\n    def _

In [4]:
# BreakDown on Task (When something is hard, we break it down)

# 1. Construct step-wise PNG
#    - transparency changing function
#    - growing strategy (code-based subnodes as importance score)
# 2. Animate into GIF (not perfect due to scale disconnection)


# 3. Agent to understand codebase with repo-DAG